# CODA-database-sandbox

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

#### Date: 11 September 2024

### import packages

In [1]:
import xarray as xr
import pandas as pd

In [2]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40719,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:38069,Total threads: 4
Dashboard: /proxy/39709/status,Memory: 35.88 GiB
Nanny: tcp://127.0.0.1:35885,


In [5]:
data_path = '/g/data/es60/users/thomas_moore/CODA/2012/'

In [6]:
ds = xr.open_dataset(data_path+ "WOD2018_CODA_2012_pfl.nc",chunks="auto")

In [7]:
ds

<xarray.Dataset> Size: 71GB
Dimensions:                 (cast: 142922, z_index: 4446)
Coordinates:
  * cast                    (cast) int64 1MB 1 2 3 4 ... 142920 142921 142922
    time                    (cast) datetime64[ns] 1MB dask.array<chunksize=(142922,), meta=np.ndarray>
    lat                     (cast) float32 572kB dask.array<chunksize=(142922,), meta=np.ndarray>
    lon                     (cast) float32 572kB dask.array<chunksize=(142922,), meta=np.ndarray>
  * z_index                 (z_index) int64 36kB 0 1 2 3 ... 4442 4443 4444 4445
    z                       (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
Data variables: (12/33)
    WOD_id                  (cast) int32 572kB dask.array<chunksize=(142922,), meta=np.ndarray>
    origflagset             (cast) |S100 14MB dask.array<chunksize=(142922,), meta=np.ndarray>
    country                 (cast) |S100 14MB dask.array<chunksize=(142922,), meta=np.ndarray>
    dataset                 (cast) |S100 14MB dask.array<chunksize=(142922,), meta=np.ndarray>
    Access_no               (cast) float64 1MB dask.array<chunksize=(142922,), meta=np.ndarray>
    dbase_orig              (cast) |S100 14MB dask.array<chunksize=(142922,), meta=np.ndarray>
    ...                      ...
    pH_WODflag              (cast, z_index) float64 5GB dask.array<chunksize=(23236, 722), meta=np.ndarray>
    pH_origflag             (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
    z_WODflag               (cast, z_index) float64 5GB dask.array<chunksize=(23236, 722), meta=np.ndarray>
    z_origflag              (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
    crs                     int32 4B ...
    CODA_id                 (cast) |S20 3MB dask.array<chunksize=(142922,), meta=np.ndarray>
Attributes:
    Parent ragged array file:  wod_pfl_2012.nc

In [21]:
ds = ds.set_coords(['WOD_id',
                    'origflagset',
                    'country',
                    'dataset',
                    'Access_no',
                    'dbase_orig',
                    'Project',
                    'WOD_cruise_identifier',
                    'Institute',
                    'Ocean_Vehicle',
                    'Temperature_Instrument',
                    'CODA_id'])

In [22]:
ds

<xarray.Dataset> Size: 71GB
Dimensions:                 (cast: 142922, z_index: 4446)
Coordinates: (12/18)
  * cast                    (cast) int64 1MB 1 2 3 4 ... 142920 142921 142922
    WOD_id                  (cast) int32 572kB dask.array<chunksize=(142922,), meta=np.ndarray>
    time                    (cast) datetime64[ns] 1MB dask.array<chunksize=(142922,), meta=np.ndarray>
    lat                     (cast) float32 572kB dask.array<chunksize=(142922,), meta=np.ndarray>
    lon                     (cast) float32 572kB dask.array<chunksize=(142922,), meta=np.ndarray>
    origflagset             (cast) |S100 14MB dask.array<chunksize=(142922,), meta=np.ndarray>
    ...                      ...
    Institute               (cast) |S100 14MB dask.array<chunksize=(142922,), meta=np.ndarray>
    Ocean_Vehicle           (cast) |S100 14MB dask.array<chunksize=(142922,), meta=np.ndarray>
    Temperature_Instrument  (cast) |S100 14MB dask.array<chunksize=(142922,), meta=np.ndarray>
  * z_index                 (z_index) int64 36kB 0 1 2 3 ... 4442 4443 4444 4445
    z                       (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
    CODA_id                 (cast) |S20 3MB dask.array<chunksize=(142922,), meta=np.ndarray>
Data variables: (12/21)
    Oxygen                  (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
    Oxygen_WODflag          (cast, z_index) float64 5GB dask.array<chunksize=(23236, 722), meta=np.ndarray>
    Oxygen_origflag         (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
    Temperature             (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
    Temperature_WODflag     (cast, z_index) float64 5GB dask.array<chunksize=(23236, 722), meta=np.ndarray>
    Temperature_origflag    (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
    ...                      ...
    pH                      (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
    pH_WODflag              (cast, z_index) float64 5GB dask.array<chunksize=(23236, 722), meta=np.ndarray>
    pH_origflag             (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
    z_WODflag               (cast, z_index) float64 5GB dask.array<chunksize=(23236, 722), meta=np.ndarray>
    z_origflag              (cast, z_index) float32 3GB dask.array<chunksize=(32863, 1021), meta=np.ndarray>
    crs                     int32 4B ...
Attributes:
    Parent ragged array file:  wod_pfl_2012.nc

In [ ]:
%%time
df = ds[['Temperature']].to_dataframe().reset_index()
df

In [16]:
%%time
# Convert byte strings to regular strings by decoding
df['Institute'] = df['Institute'].str.decode('utf-8')

In [18]:
df

,cast,z_index,Temperature,WOD_id,time,lat,lon,Institute,z
0,1,0,25.733000,15155888,2012-01-01 00:00:00,19.489,63.230999,U.S. NAVY,4.373211
1,1,1,25.733000,15155888,2012-01-01 00:00:00,19.489,63.230999,U.S. NAVY,9.541431
2,1,2,25.733000,15155888,2012-01-01 00:00:00,19.489,63.230999,U.S. NAVY,14.311981
3,1,3,25.745001,15155888,2012-01-01 00:00:00,19.489,63.230999,U.S. NAVY,19.479952
4,1,4,25.760000,15155888,2012-01-01 00:00:00,19.489,63.230999,U.S. NAVY,24.250269
...,...,...,...,...,...,...,...,...,...
635431207,142922,4441,NaN,15599921,2012-12-31 23:15:00,21.968,131.645996,JAPAN METEOROLOGICAL AGENCY (JMA),NaN
635431208,142922,4442,NaN,15599921,2012-12-31 23:15:00,21.968,131.645996,JAPAN METEOROLOGICAL AGENCY (JMA),NaN
635431209,142922,4443,NaN,15599921,2012-12-31 23:15:00,21.968,131.645996,JAPAN METEOROLOGICAL AGENCY (JMA),NaN
635431210,142922,4444,NaN,15599921,2012-12-31 23:15:00,21.968,131.645996,JAPAN METEOROLOGICAL AGENCY (JMA),NaN


In [20]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 635431212 entries, 0 to 635431211
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   cast         int64         
 1   z_index      int64         
 2   Temperature  float32       
 3   WOD_id       int32         
 4   time         datetime64[ns]
 5   lat          float32       
 6   lon          float32       
 7   Institute    object        
 8   z            float32       
dtypes: datetime64[ns](1), float32(4), int32(1), int64(2), object(1)
memory usage: 65.5 GB


# list of todo's

create parquet from a df
consider utility of turning "metadata" vars into coords
consider removing NaN's at df step
consider removing NaN's otherwise



In [ ]:
df.dropna(how='all',column='z')

In [ ]:
%%time
df = ds.to_dataframe()#.reset_index()

In [ ]:
%%time
df.to_parquet(data_path+"my_file.parquet")